In [1]:
import pandas as pd

In [2]:
INPUT_DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUTPUT_DATA_DIR = "../../data/processed/"
OUTPUT_FILE_NAME = 'squashed_tags.parquet'

df = pd.read_parquet(INPUT_DATA_DIR + INPUT_FILE_NAME)

In [3]:
df['tags'][0]

'cars,alternative energy,culture,politics,science,climate change,environment,sustainability,global issues,technology'

In [4]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()

# df['tags'] = split_tags

tag_cutoff = int(0.01*len(df.index))
print(tag_cutoff)
squashed_tags = pd.DataFrame(tag_counts)
# squashed_tags.columns = ['tag','count']
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)
print(len(squash_list))

23
179


In [5]:
for i in range(len(df.index)):
    original_tags = df['tags'][i]
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    df['tags'][i] = final_string

/home/kayes/anaconda3/envs/cds/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
df[df['tags']=='']

,speaker,headline,description,duration,tags,transcript,WC


In [7]:
df['tags']

0       alternative energy,culture,politics,science,cl...
1       industrial design,alternative energy,invention...
2       poverty,economics,investment,culture,politics,...
3       industrial design,invention,engineering,entrep...
4       industrial design,transportation,invention,des...
                              ...                        
2381    architecture,beauty,cities,community,journalis...
2382    ai,algorithm,brain,cognitive science,mental he...
2383    choice,fear,goal-setting,humanity,life,persona...
2384    aging,art,beauty,books,communication,family,hu...
2385    adventure,collaboration,demo,design,engineerin...
Name: tags, Length: 2386, dtype: object

In [8]:
df.to_parquet(OUTPUT_DATA_DIR + OUTPUT_FILE_NAME)